In [ ]:
import numpy as np
import pandas as pd
import main

rigis = pd.read_csv(main.data_dir + "\\rigis.csv", delimiter=';')
del rigis['Пласт РИГИС']
del rigis['Пластпропласт']
del rigis['Литология']
del rigis['Коллектор']

rigis = rigis.rename(columns={'Скважина': 'well_name'})
rigis = rigis.rename(columns={'Абсолютная глубина': 'z_abs'})
rigis = rigis.rename(columns={'Абсолютная толщина': 'h_abs'})
rigis = rigis.rename(columns={'Глубина кровли пласта': 'z'})
rigis = rigis.rename(columns={'Толщина': 'h'})

rigis.head()

FileNotFoundError: File b'..\\tasks\\task 6\\Data\\rigis.csv' does not exist

In [2]:
rigis.replace(' ', '', regex=True, inplace=True)
rigis['z_abs'] = rigis['z_abs'].astype('float64')
rigis['h_abs'] = rigis['h_abs'].astype('float64')
rigis['z'] = rigis['z'].astype('float64')
rigis['h'] = rigis['h'].astype('float64')
rigis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3688 entries, 0 to 3687
Data columns (total 5 columns):
well_name    3688 non-null object
z            3688 non-null float64
h            3688 non-null float64
z_abs        3688 non-null float64
h_abs        3687 non-null float64
dtypes: float64(4), object(1)
memory usage: 144.1+ KB


In [3]:
header_data = pd.read_csv(main.data_dir + "\Well_Header.csv", delimiter='\t')
header_data.head()
header_data = header_data.rename(columns={'WellName': 'well_name'})
del header_data['KB']
del header_data['TopDepth']
del header_data['BottomDepth']
del header_data['Symbol']
del header_data['Rig']
del header_data['Field']
header_data.head()

,well_name,X-Coord,Y-Coord
0,314_103ST2,354719.52,6470051.42
1,314_107,354693.18,6470164.36
2,314_121,354433.25,6468384.38
3,314_125PL,354717.25,6470061.16
4,314_134,354478.44,6468306.61


In [5]:
header_data['well_name'].replace('314_', '', regex=True, inplace=True)
header_data.replace(' ', '', regex=True, inplace=True)
header_data['X-Coord'] = header_data['X-Coord'].astype('float64')
header_data['Y-Coord'] = header_data['Y-Coord'].astype('float64')
header_data.head()

,well_name,X-Coord,Y-Coord
0,103ST2,354719.52,6470051.42
1,107,354693.18,6470164.36
2,121,354433.25,6468384.38
3,125PL,354717.25,6470061.16
4,134,354478.44,6468306.61


In [6]:
rigis = pd.merge(rigis, header_data, on='well_name')
rigis.head()

,well_name,z,h,z_abs,h_abs,X-Coord,Y-Coord
0,334,2748.6,1.0,2530.41,1.0,351562.55,6464575.94
1,334,2749.6,1.2,2531.41,1.2,351562.55,6464575.94
2,334,2750.8,0.4,2532.61,0.4,351562.55,6464575.94
3,334,2751.2,0.4,2533.01,0.4,351562.55,6464575.94
4,334,2751.6,1.0,2533.41,1.0,351562.55,6464575.94


In [7]:
top = rigis.groupby(['well_name'])['z'].min()

In [8]:
top_df =  top.to_frame().reset_index()
top_df.head()

,well_name,z
0,100,2709.4
1,101,3156.4
2,102,2792.8
3,103,2895.0
4,103ST2,2888.4


In [9]:
top_df = pd.merge(top_df, header_data, on='well_name')
top_df.head()
top_df.shape

(231, 4)

In [12]:
top_df.head()

,well_name,z,X-Coord,Y-Coord
0,100,2709.4,354727.38,6470017.85
1,101,3156.4,354708.18,6470100.11
2,102,2792.8,354711.60,6470085.50
3,103,2895.0,354719.52,6470051.42
4,103ST2,2888.4,354719.52,6470051.42


In [14]:
top_df.to_csv('top_df.csv', sep =';', columns = ['well_name', 'X-Coord', 'Y-Coord', 'z'], index = False, header =False,
             line_terminator = ';\n')

In [16]:
botom = rigis.groupby(['well_name'])['z'].max()
botom_df =  botom.to_frame().reset_index()
botom_df.head()

,well_name,z
0,100,2724.2
1,101,3170.8
2,102,2805.0
3,103,2910.0
4,103ST2,2905.6


In [17]:
rigis_h = rigis[['well_name', 'z', 'h']]
botom_df =  pd.merge(botom_df, rigis_h, on=['well_name', 'z'])
botom_df.head()

,well_name,z,h
0,100,2724.2,1.2
1,101,3170.8,1.2
2,102,2805.0,1.6
3,103,2910.0,1.6
4,103ST2,2905.6,3.0


In [18]:
botom_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 231 entries, 0 to 230
Data columns (total 3 columns):
well_name    231 non-null object
z            231 non-null float64
h            231 non-null float64
dtypes: float64(2), object(1)
memory usage: 7.2+ KB


In [19]:
botom_df['z'] = botom_df['z'] + botom_df['h']

In [20]:
del botom_df['h']
botom_df.head()

,well_name,z
0,100,2725.4
1,101,3172.0
2,102,2806.6
3,103,2911.6
4,103ST2,2908.6


In [21]:
botom_df = pd.merge(botom_df, header_data, on='well_name')
botom_df.head()
# bottom_df.shape

,well_name,z,X-Coord,Y-Coord
0,100,2725.4,354727.38,6470017.85
1,101,3172.0,354708.18,6470100.11
2,102,2806.6,354711.60,6470085.50
3,103,2911.6,354719.52,6470051.42
4,103ST2,2908.6,354719.52,6470051.42


In [22]:
botom_df.to_csv('bottom_df.csv', sep =';', columns = ['well_name', 'X-Coord', 'Y-Coord', 'z'], index = False, header =False,
             line_terminator = ';\n')